In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt
from math import log2
%matplotlib inline

from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.pipeline import make_pipeline
import time
import matplotlib as mpl

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor



from sklearn.preprocessing import StandardScaler,PowerTransformer,MinMaxScaler,QuantileTransformer,normalize
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold, SelectKBest
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

import pathlib
import os
import sys

# for selection the right path
import os,sys,inspect,pathlib
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

from common.model_trainer_reg import *

In [ ]:
# Custom functions

module_path = pathlib.Path(os.getcwd()).parent
if str(module_path) not in sys.path:
    sys.path.append(str(module_path))
print(sys.path)

from common.dataset_grabber import get_data_path

In [ ]:
datapath = get_data_path("Metro", "Metro_Interstate_Traffic_Volume.csv")
display(datapath)

In [ ]:
df_raw = pd.read_csv(datapath)

df_raw

# Dataset preparation

In [ ]:
df_raw.info()

# different holiday

In [ ]:
df_raw

In [ ]:
holiday_list = df_raw.holiday.unique()
holiday_list

In [ ]:
for l in holiday_list:
    if l == "None":
        df_raw = df_raw.replace(l,0)
for l in holiday_list:
    if l != "None":
        df_raw = df_raw.replace(l,1)

In [ ]:
df_raw


In [ ]:
holiday_list = df_raw.holiday.unique()
holiday_list

In [ ]:
indexNames = df_raw.index[df_raw.holiday == 1]
df_raw.drop(indexNames , inplace=True)

In [ ]:
df_raw

# handling the outlayers
## some distributions

The minimal temperature in one row is 0.0 Kelvin that is not possible. So the row will delitet and so the one row with the 9831.3 rain.

In [ ]:
sns.distplot(df_raw["temp"], color="red")
plt.show()
sns.distplot(df_raw["rain_1h"], color="red")
plt.show()
sns.distplot(df_raw["snow_1h"], color="red")
plt.show()

## show the outliers

In [ ]:
print("min Ttemperatur K: ",np.min(df_raw["temp"]),"mean Ttemperatur K: ",np.mean(df_raw["temp"]))
print("max rain: ",np.max(df_raw["rain_1h"]),"mean rain: ",np.mean(df_raw["rain_1h"]))
print("max snow: ",np.max(df_raw["snow_1h"]),"max snow: ",np.min(df_raw["snow_1h"]))

## clean the dataset by dropping the rows with the outlires

In [ ]:
indexNames = df_raw.index[df_raw.temp == np.min(df_raw["temp"])]
df_raw.drop(indexNames , inplace=True)

In [ ]:
print("min Ttemperatur K: ",np.min(df_raw["temp"]),"mean Ttemperatur K: ",np.mean(df_raw["temp"]))
print("max rain: ",np.max(df_raw["rain_1h"]),"mean rain: ",np.mean(df_raw["rain_1h"]))
print("max snow: ",np.max(df_raw["snow_1h"]),"max snow: ",np.min(df_raw["snow_1h"]))

In [ ]:
indexNames = df_raw.index[df_raw.rain_1h > 300]
df_raw.drop(indexNames , inplace=True)

In [ ]:
print("min Ttemperatur K: ",np.min(df_raw["temp"]),"mean Ttemperatur K: ",np.mean(df_raw["temp"]))
print("max rain: ",np.max(df_raw["rain_1h"]),"mean rain: ",np.mean(df_raw["rain_1h"]))
print("max snow: ",np.max(df_raw["snow_1h"]),"max snow: ",np.min(df_raw["snow_1h"]))

## looks much better

In [ ]:
sns.distplot(df_raw["temp"], color="red")
plt.show()
sns.distplot(df_raw["rain_1h"], color="red")
plt.show()
sns.distplot(df_raw["snow_1h"], color="red")
plt.show()

## converts the date coloum into a better fromat

In [ ]:
df_raw["date_time"] = pd.to_datetime(df_raw.date_time)
df_raw["hour"] = df_raw.date_time.dt.hour
df_raw["day"] = df_raw.date_time.dt.day
df_raw["month"] = df_raw.date_time.dt.month
df_raw["year"] = df_raw.date_time.dt.year

df_raw = df_raw.drop("date_time", axis=1)

df_raw

## code the weather discription into numbers


In [ ]:
lsitweather = df_raw.weather_main.unique()
l = 0
for j in lsitweather:
    df_raw = df_raw.replace(j,l)
    l = l + 1

lsitweather = df_raw.weather_description.unique()
l = 0
for j in lsitweather:
    df_raw = df_raw.replace(j,l)
    l = l + 1


In [ ]:
df_raw

# Prepare it for the training

In [ ]:
#cols_to_drop = ['holiday', 'weather_main', 'weather_description']
#cols_to_drop = ['holiday', 'weather_main', 'weather_description']
#df_prep = df_raw.drop(cols_to_drop, axis=1)

In [ ]:
df_prep = df_raw

In [ ]:
np.mean(df_prep["traffic_volume"])
plot = sns.distplot(df_prep["traffic_volume"], color="red")

# Train and Test

## SGD-Regression

In [ ]:
Y = df_prep[['traffic_volume']]
X = df_prep[["temp","rain_1h","snow_1h","clouds_all","hour","day","month","year","weather_main","weather_description"]]

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [ ]:
MODEL = SGDRegressor
params = {"alpha" : [0.0001]}

In [ ]:
modeltrainer = ModelTrainer(MODEL, params, X, Y, r2_score, StandardScaler, train_test_split, KFold, thread_cnt=4)
########### train with TrainTestSplit  ###################
modeltrainer.TTSplit(perc = 0.7)
modeltrainer.train()


########### k-Fold Cross validation  ###################
modeltrainer.CV_fold(k = 6)
modeltrainer.train()

## KNN-Regression

In [ ]:
from KNN.KNNRegressor import KNNRegressor


In [ ]:
MODEL = KNNRegressor
params = {"weights" : ["uniform"]}

In [ ]:
modeltrainer = ModelTrainer(MODEL, params, X, Y, r2_score, StandardScaler, train_test_split, KFold, thread_cnt=4)
########### train with TrainTestSplit  ###################
modeltrainer.TTSplit(perc = 0.2)
modeltrainer.train()


########### k-Fold Cross validation  ###################
#modeltrainer.CV_fold(k = 6)
#modeltrainer.train()

## RF-Regression

In [ ]:
MODEL = RandomForestRegressor
params = {"n_estimators" : [100]}

In [ ]:
modeltrainer = ModelTrainer(MODEL, params, X, Y, r2_score, StandardScaler, train_test_split, KFold, thread_cnt=4)
########### train with TrainTestSplit  ###################
modeltrainer.TTSplit(perc = 0.7)
modeltrainer.train()


########### k-Fold Cross validation  ###################
modeltrainer.CV_fold(k = 6)
modeltrainer.train()

## DT-Regression

In [ ]:
MODEL = DecisionTreeRegressor
params = {"criterion": ["mse"]}

In [ ]:
modeltrainer = ModelTrainer(MODEL, params, X, Y, r2_score, StandardScaler, train_test_split, KFold, thread_cnt=4)
########### train with TrainTestSplit  ###################
modeltrainer.TTSplit(perc = 0.7)
modeltrainer.train()


########### k-Fold Cross validation  ###################
modeltrainer.CV_fold(k = 6)
modeltrainer.train()